In [282]:
import nilearn
import nibabel as nib
import nltools.prefs as prf
from nilearn import datasets, image, input_data

import sys
import pandas as pd

import os
import glob

%matplotlib inline
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 500)
pd.set_option('display.width', 1000)

from IPython.core.display import display, HTML
display(HTML("<style>"
    + "#notebook { padding-top:0px !important; } " 
    + ".container { width:100% !important; } "
    + ".end_space { min-height:0px !important; } "
    + "</style>"))

In [283]:
# Load Ng images from neurovault
# Read Keren image
# _neg refers to a negative contrast value of MDD - HV, so MDD < HV
# _pos refers to MDD > HV

from nilearn.datasets import fetch_neurovault_ids
ng = fetch_neurovault_ids(collection_ids=[3884]) 

image.load_img(ng['images'][0]).to_filename('/data/MBDU/midla/notebooks/campcc/midla_review_dat/ng_pos.nii.gz')
image.load_img(ng['images'][2]).to_filename('/data/MBDU/midla/notebooks/campcc/midla_review_dat/ng_neg1.nii.gz')
image.load_img(ng['images'][4]).to_filename('/data/MBDU/midla/notebooks/campcc/midla_review_dat/ng_neg2.nii.gz')

keren_neg = image.load_img('/data/MBDU/midla/notebooks/campcc/midla_review_dat/ALEtxt_feedback_MDD+HR+ContvsHV_final_woROI_wsmoski2011_wJon_ALE_C05_1_mni152_thresh.nii.gz')

Reading local neurovault data.
Already fetched 1 image
Already fetched 2 images
Already fetched 3 images
Already fetched 4 images
Already fetched 5 images
Already fetched 6 images
Already fetched 7 images
Already fetched 8 images
Already fetched 9 images
9 images found on local disk.


In [284]:
# Read Zhang coordinates
zhang = pd.read_csv('/data/MBDU/midla/notebooks/campcc/midla_review_dat/zhang_coords.csv')
zhang_neg = zhang.loc[zhang['increased/decreased'] == 'decreased']
zhang_pos = zhang.loc[zhang['increased/decreased'] == 'increased']

In [285]:
# Create blank brain images
! module load afni; 3dcalc -a /data/MBDU/midla/notebooks/campcc/midla_review_dat/TT_N27+tlrc.HEAD -expr 'a * 0' -prefix ball_pos.nii.gz -overwrite
! module load afni; 3dcalc -a /data/MBDU/midla/notebooks/campcc/midla_review_dat/TT_N27+tlrc.HEAD -expr 'a * 0' -prefix ball_neg.nii.gz -overwrite

[+] Loading AFNI current-openmp  ... 
AFNI/current-openmp last updated 2020-05-15

++ 3dcalc: AFNI version=AFNI_20.1.06 (May  4 2020) [64-bit]
++ Authored by: A cast of thousands
*+ WARNING: output sub-brick 0 is all zeros!
++ Output dataset ./ball_pos.nii.gz
[+] Loading AFNI current-openmp  ... 
AFNI/current-openmp last updated 2020-05-15

++ 3dcalc: AFNI version=AFNI_20.1.06 (May  4 2020) [64-bit]
++ Authored by: A cast of thousands
*+ WARNING: output sub-brick 0 is all zeros!
++ Output dataset ./ball_neg.nii.gz


In [286]:
# Project Zhang coordinates onto brain images
for _,row in zhang_pos.iterrows():
    program = 'module load afni; 3dcalc'
    temp = '/data/MBDU/midla/notebooks/campcc/ball_pos.nii.gz'
    rsquar = (((float(row.volume)*3.0)/(3.14*4.)) ** (1.0/3.0)) ** 2
    expr = f'step(a+step({rsquar}-(x-{-1*row.x})*(x-{-1*row.x})-(y-{-1*row.y})*(y-{-1*row.y})-(z-{row.z})*(z-{row.z})))'
    prefix = f'ball_pos.nii.gz'
    runline = f"{program} -a {temp} -expr \"{expr}\" -prefix {prefix} -overwrite"
    if row.volume > 0:
        ! $runline
        
for _,row in zhang_neg.iterrows():
    program = 'module load afni; 3dcalc'
    temp = '/data/MBDU/midla/notebooks/campcc/ball_neg.nii.gz'
    rsquar = (((float(row.volume)*3.0)/(3.14*4.)) ** (1.0/3.0)) ** 2
    expr = f'step(a+step({rsquar}-(x-{-1*row.x})*(x-{-1*row.x})-(y-{-1*row.y})*(y-{-1*row.y})-(z-{row.z})*(z-{row.z})))'
    prefix = f'ball_neg.nii.gz'
    runline = f"{program} -a {temp} -expr \"{expr}\" -prefix {prefix} -overwrite"
    if row.volume > 0:
        ! $runline

[+] Loading AFNI current-openmp  ... 
AFNI/current-openmp last updated 2020-05-15

++ 3dcalc: AFNI version=AFNI_20.1.06 (May  4 2020) [64-bit]
++ Authored by: A cast of thousands
++ Symbol x using predefined value
++ Symbol y using predefined value
++ Symbol z using predefined value
++ Output dataset ./ball_pos.nii.gz
[+] Loading AFNI current-openmp  ... 
AFNI/current-openmp last updated 2020-05-15

++ 3dcalc: AFNI version=AFNI_20.1.06 (May  4 2020) [64-bit]
++ Authored by: A cast of thousands
++ Symbol x using predefined value
++ Symbol y using predefined value
++ Symbol z using predefined value
++ Output dataset ./ball_pos.nii.gz
[+] Loading AFNI current-openmp  ... 
AFNI/current-openmp last updated 2020-05-15

++ 3dcalc: AFNI version=AFNI_20.1.06 (May  4 2020) [64-bit]
++ Authored by: A cast of thousands
++ Symbol x using predefined value
++ Symbol y using predefined value
++ Symbol z using predefined value
++ Output dataset ./ball_pos.nii.gz
[+] Loading AFNI current-openmp  ... 
AF

++ Output dataset ./ball_neg.nii.gz
[+] Loading AFNI current-openmp  ... 
AFNI/current-openmp last updated 2020-05-15

++ 3dcalc: AFNI version=AFNI_20.1.06 (May  4 2020) [64-bit]
++ Authored by: A cast of thousands
++ Symbol x using predefined value
++ Symbol y using predefined value
++ Symbol z using predefined value
++ Output dataset ./ball_neg.nii.gz
[+] Loading AFNI current-openmp  ... 
AFNI/current-openmp last updated 2020-05-15

++ 3dcalc: AFNI version=AFNI_20.1.06 (May  4 2020) [64-bit]
++ Authored by: A cast of thousands
++ Symbol x using predefined value
++ Symbol y using predefined value
++ Symbol z using predefined value
++ Output dataset ./ball_neg.nii.gz
[+] Loading AFNI current-openmp  ... 
AFNI/current-openmp last updated 2020-05-15

++ 3dcalc: AFNI version=AFNI_20.1.06 (May  4 2020) [64-bit]
++ Authored by: A cast of thousands
++ Symbol x using predefined value
++ Symbol y using predefined value
++ Symbol z using predefined value
++ Output dataset ./ball_neg.nii.gz
[+] 

In [287]:
# Transform images to MNI space
!module load afni; 3dNwarpApply -nwarp /data/MBDU/nielsond/awpy/anat.un.aff.qw_WARP.nii /data/MBDU/nielsond/awpy/anat.un.aff.Xat.1D -source ball_pos.nii.gz -interp NN -overwrite -master /data/MBDU/nielsond/awpy/MNI152_T1_1mm_brain.nii.gz -prefix ball_pos_mni.nii.gz -overwrite
!module load afni; 3dNwarpApply -nwarp /data/MBDU/nielsond/awpy/anat.un.aff.qw_WARP.nii /data/MBDU/nielsond/awpy/anat.un.aff.Xat.1D -source ball_neg.nii.gz -interp NN -overwrite -master /data/MBDU/nielsond/awpy/MNI152_T1_1mm_brain.nii.gz -prefix ball_neg_mni.nii.gz -overwrite

[+] Loading AFNI current-openmp  ... 
AFNI/current-openmp last updated 2020-05-15

++ 3dNwarpApply: AFNI version=AFNI_20.1.06 (May  4 2020) [64-bit]
++ Authored by: Zhark the Warped
++ opened source dataset 'ball_pos.nii.gz'
++ -master dataset is '/data/MBDU/nielsond/awpy/MNI152_T1_1mm_brain.nii.gz'
++ Processing -nwarp 
++ Warping:.Z
++ Output dataset ./ball_pos_mni.nii.gz
++ total Elapsed time = 2.3 sec
[+] Loading AFNI current-openmp  ... 
AFNI/current-openmp last updated 2020-05-15

++ 3dNwarpApply: AFNI version=AFNI_20.1.06 (May  4 2020) [64-bit]
++ Authored by: Zhark the Warped
++ opened source dataset 'ball_neg.nii.gz'
++ -master dataset is '/data/MBDU/nielsond/awpy/MNI152_T1_1mm_brain.nii.gz'
++ Processing -nwarp 
++ Warping:.Z
++ Output dataset ./ball_neg_mni.nii.gz
++ total Elapsed time = 2.2 sec
